In [ ]:
# 标准库导入
import os
import sys
import time
import yaml
import random
import logging
import pickle
import json
from pathlib import Path
from typing import Dict, Any, Optional, List, Tuple

# 第三方库导入
from torch.utils.data import DataLoader
import torch.nn.functional as F
from torch.optim.lr_scheduler import CosineAnnealingLR, StepLR, ReduceLROnPlateau
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import f1_score, confusion_matrix, classification_report
from sklearn.utils import class_weight
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns

# 项目内部导入
try:
    from config.config_loader import ConfigLoader
    from config.config_bridge import ConfigBridge
except ImportError:
    print("警告: 配置模块未找到，将只使用传统模式")
    ConfigLoader = None
    ConfigBridge = None

import utils_torch as utils
import model_cbranchformer as model

print("所有模块导入完成")

In [ ]:
# === 从 utils_torch.py 移入的关键代码 ===
class HARDataset(torch.utils.data.Dataset):
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        return self.data[idx], self.labels[idx]

# === 配置桥接器 (简化版) ===
class DotDict(dict):
    """支持点号访问的字典"""
    def __init__(self, data):
        super().__init__()
        self._dict = {}
        for key, value in data.items():
            self._dict[key] = self._to_dot_notation(value)
            setattr(self, key, self._dict[key])
    
    def keys(self):
        return self._dict.keys()
    
    def items(self):
        return [(k, getattr(self, k)) for k in self._dict.keys()]
    
    def get(self, key, default=None):
        return getattr(self, key, default)
    
    def _to_dot_notation(self, data):
        if isinstance(data, dict):
            return DotDict(data)
        elif isinstance(data, list):
            return [self._to_dot_notation(i) for i in data]
        else:
            return data

class SimplifiedConfigBridge:
    """简化的配置桥接器"""
    def __init__(self, config_path: str = None):
        self.config_path = config_path
        self.raw_config = {}
        
        if config_path and os.path.exists(config_path):
            with open(config_path, 'r', encoding='utf-8') as f:
                self.raw_config = yaml.safe_load(f)
        
        self.config = self._to_dot_notation(self.raw_config)
    
    def _to_dot_notation(self, data):
        if isinstance(data, dict):
            return DotDict(data)
        elif isinstance(data, list):
            return [self._to_dot_notation(i) for i in data]
        else:
            return data

    def get_dataset_config(self) -> Dict[str, Any]:
        return self.raw_config.get('dataset', {})

    def get_training_config(self) -> Dict[str, Any]:
        return self.raw_config.get('training', {})
        
    def get_visualization_config(self) -> Dict[str, Any]:
        return self.raw_config.get('visualization', {})

print("✓ 辅助工具和类定义完成")

In [ ]:
class ConfigurableTrainer:
    """配置驱动的训练器类"""
    
    def __init__(self, config_path: Optional[str] = None):
        """初始化训练器"""
        self.config_path = config_path
        self.use_config = config_path is not None and os.path.exists(config_path)
        
        # 初始化配置桥接器
        if self.use_config:
            try:
                self.config_bridge = SimplifiedConfigBridge(config_path)
                print(f"✓ 配置文件加载成功: {config_path}")
            except Exception as e:
                print(f"❌ 配置文件加载失败: {e}")
                self.use_config = False
        
        # 初始化其他属性
        self.model = None
        self.optimizer = None
        self.scheduler = None
        self.criterion = None
        self.device = None
        self.logger = None
        self.gradient_clip_norm = 0
        self.output_dir = None
    
    def get_params(self) -> Dict[str, Any]:
        """获取训练参数"""
        if self.use_config:
            return self._get_config_parameters()
        else:
            return self._get_hardcoded_parameters()
    
    def _get_config_parameters(self) -> Dict[str, Any]:
        """从配置文件获取参数"""
        config = self.config_bridge.raw_config
        
        params = {
            # 基本设置
            'device': config.get('device', 'auto'),
            'random_seed': config.get('seed', 42),
            'output_dir': config.get('output_dir', './results/shl_multimodal'),
            'save_checkpoints': config.get('save_checkpoints', True),
            'verbose': config.get('verbose', True),
            
            # 数据集配置
            'dataset_name': config.get('dataset', {}).get('name', 'SHL'),
            'data_path': config.get('dataset', {}).get('path', './datasets/'),
            
            # 训练配置
            'batch_size': config.get('training', {}).get('batch_size', 32),
            'learning_rate': config.get('training', {}).get('learning_rate', 0.001),
            'epochs': config.get('training', {}).get('epochs', 100),
            'weight_decay': config.get('training', {}).get('weight_decay', 0.0001),
            'gradient_clip_norm': config.get('training', {}).get('gradient_clip_norm', 1.0),
            'early_stopping_patience': config.get('training', {}).get('early_stopping_patience', 10),
        }
        
        return params
    
    def setup_environment(self, params: Dict[str, Any]) -> None:
        """设置训练环境"""
        try:
            # 设置随机种子
            seed = params['random_seed']
            random.seed(seed)
            np.random.seed(seed)
            torch.manual_seed(seed)
            if torch.cuda.is_available():
                torch.cuda.manual_seed(seed)
                torch.cuda.manual_seed_all(seed)
            
            # 设置设备
            device_config = params['device']
            if device_config == 'auto':
                if torch.cuda.is_available():
                    self.device = torch.device('cuda')
                    print(f"使用 CUDA 设备: {torch.cuda.get_device_name()}")
                elif torch.backends.mps.is_available():
                    self.device = torch.device('mps')
                    print("使用 MPS 设备")
                else:
                    self.device = torch.device('cpu')
                    print("使用 CPU 设备")
            else:
                self.device = torch.device(device_config)
                print(f"使用指定设备: {self.device}")
            
            # 设置输出目录
            self.output_dir = Path(params['output_dir'])
            self.output_dir.mkdir(parents=True, exist_ok=True)
            
            # 保存梯度裁剪参数
            self.gradient_clip_norm = params.get('gradient_clip_norm', 0)
            
            # PyTorch 性能优化
            torch.backends.cudnn.benchmark = True
            torch.backends.cudnn.deterministic = False
            
            print("✓ 环境设置完成")
            
        except Exception as e:
            print(f"❌ 环境设置失败: {e}")
            raise
    
    def load_data(self, params: Dict[str, Any]) -> Tuple[DataLoader, DataLoader, DataLoader]:
        """加载数据 - 占位符方法"""
        # 这个方法在实际使用中会被外部数据加载逻辑替代
        print("⚠️  使用占位符数据加载方法")
        return None, None, None
    
    def train(self):
        """执行训练"""
        print("开始配置驱动训练...")
        
        # 获取参数并设置环境
        params = self.get_params()
        self.setup_environment(params)
        
        print("✓ 配置驱动训练器初始化完成")
        print("   实际训练将由后续cells执行")

print("✓ ConfigurableTrainer类定义完成")

In [ ]:
# ========== 配置生成 - 支持SHL_Multimodal数据集 ==========
# 🔥 每次训练只需要修改这个cell中的參數

# === 基本配置参数 ===
USE_CONFIG_MODE = False  # True=配置驱动模式, False=传统模式 (推荐False以确保稳定性)
CONFIG_PATH = "config/default_configs/shl_config.yaml"  # SHL数据集配置文件路径

# === SHL_Multimodal数据集配置 ===
dataset_name = "SHL"  # 数据集名称
# 🔥 修改为您的实际文件路径
SHL_DATA_PATH = "/Users/zilongzeng/Research/MazeruHAR/datasets/datasetStandardized/SHL_Multimodal"

data_config = {
    'window_size': 128,
    'step_size': 64,
    'normalize': True,
    'filter_freq': 20,
    'modalities': ['imu', 'magnetometer']  # 启用的模态
}

# === 🔥 主要训练配置 (经常修改的参数) ===
TRAINING_CONFIG = {
    # 训练基本参数
    'batch_size': 32,               # 🔥 批次大小: 16, 32, 64
    'learning_rate': 0.001,         # 🔥 学习率: 0.0001, 0.001, 0.01
    'epochs': 10,                  # 🔥 训练轮数: 50, 100, 150, 200
    'weight_decay': 0.0001,         # 权重衰减
    'dropout_rate': 0.1,            # Dropout率: 0.1, 0.2, 0.3
    'label_smoothing': 0.1,         # 标签平滑
    'gradient_clip_norm': 1.0,      # 梯度裁剪
    'early_stopping_patience': 10, # 早停耐心值: 5, 10, 15
    
    # 模型架构参数
    'projection_dim': 192,          # 🔥 投影维度: 128, 192, 256
    'num_heads': 4,                 # 🔥 注意力头数: 4, 6, 8
    'num_layers': 3,                # 🔥 层数: 2, 3, 4
    'patch_size': 16,               # 补丁大小: 8, 16, 32
    'time_step': 16,                # 时间步长: 8, 16, 32
    'conv_kernel_size': 15,         # 卷积核大小: 7, 15, 31
}

# === 传统模式配置 ===
traditional_config = {
    # 数据集相关
    'dataset_name': dataset_name,
    'data_path': SHL_DATA_PATH,  # 使用您的实际路径
    'window_size': 128,
    'step_size': 64,
    'sample_rate': 100,
    
    # 合并训练配置
    **TRAINING_CONFIG,
    
    # 其他设置
    'device': 'auto',               # 'auto', 'cpu', 'cuda', 'mps'
    'random_seed': 42,              # 随机种子
    'output_dir': './results/shl_multimodal',
    'save_checkpoints': True,
    'verbose': True,
    'position_device': '',          # SHL数据集不使用位置设备分割
    'main_dir': './'
}

# === SHL_Multimodal数据集特定配置 ===
shl_specific_config = {
    'activity_labels': [
        'Still', 'Walking', 'Run', 'Bike', 
        'Car', 'Bus', 'Train', 'Subway'
    ],
    'modalities': {
        'imu': {
            'channels': 6,  # Accelerometer (3) + Gyroscope (3)
            'enabled': True,  # 🔥 是否启用IMU模态
            'preprocessing': {
                'normalize': True,
                'filter_freq': 20
            }
        },
        'magnetometer': {
            'channels': 3,  # Magnetometer
            'enabled': True,  # 🔥 是否启用磁力计模态
            'preprocessing': {
                'normalize': True
            }
        },
        'linear_acc': {
            'channels': 3,  # Linear Acceleration
            'enabled': False,  # 🔥 可以根据需要启用
            'preprocessing': {
                'normalize': True
            }
        },
        'gravity': {
            'channels': 3,  # Gravity
            'enabled': False,  # 🔥 可以根据需要启用
            'preprocessing': {
                'normalize': True
            }
        },
        'orientation': {
            'channels': 3,  # Orientation
            'enabled': False,  # 🔥 可以根据需要启用
            'preprocessing': {
                'normalize': True
            }
        }
    },
    'train_split': 0.7,
    'val_split': 0.15,
    'test_split': 0.15
}

# === 🔥 快速配置预设 (取消注释使用) ===
# 快速测试配置
# TRAINING_CONFIG.update({
#     'batch_size': 64,
#     'learning_rate': 0.01,
#     'epochs': 20,
#     'early_stopping_patience': 5
# })

# 精细训练配置
# TRAINING_CONFIG.update({
#     'batch_size': 16,
#     'learning_rate': 0.0005,
#     'epochs': 200,
#     'early_stopping_patience': 20
# })

# 大模型配置
# TRAINING_CONFIG.update({
#     'projection_dim': 256,
#     'num_heads': 8,
#     'num_layers': 4,
#     'batch_size': 16  # 减小批次大小
# })

# === 配置验证与初始化 ===
if USE_CONFIG_MODE and ConfigBridge is not None:
    print("✓ 使用配置驱动模式")
    print(f"  配置文件路径: {CONFIG_PATH}")
    
    # 检查配置文件是否存在
    if not os.path.exists(CONFIG_PATH):
        print(f"⚠️  配置文件不存在: {CONFIG_PATH}")
        print("   创建默认SHL_Multimodal配置文件...")
        
        # 创建默认SHL_Multimodal配置
        default_shl_config = {
            'name': 'SHL_MultiModal_Experiment',
            'dataset': {
                'name': 'SHL',
                'path': SHL_DATA_PATH,
                **shl_specific_config
            },
            'architecture': {
                'experts': {
                    'imu': {
                        'type': 'cbranchformer',
                        'params': {
                            'projection_dim': TRAINING_CONFIG['projection_dim'],
                            'num_heads': TRAINING_CONFIG['num_heads'],
                            'num_layers': TRAINING_CONFIG['num_layers'],
                            'patch_size': TRAINING_CONFIG['patch_size'],
                            'time_step': TRAINING_CONFIG['time_step'],
                            'conv_kernel_size': TRAINING_CONFIG['conv_kernel_size'],
                            'dropout_rate': TRAINING_CONFIG['dropout_rate'],
                            'output_dim': TRAINING_CONFIG['projection_dim']
                        }
                    },
                    'magnetometer': {
                        'type': 'lstm',
                        'params': {
                            'hidden_dim': 64,
                            'num_layers': 2,
                            'dropout_rate': TRAINING_CONFIG['dropout_rate'],
                            'bidirectional': True,
                            'output_dim': 64
                        }
                    }
                },
                'fusion': {
                    'strategy': 'concatenate',
                    'params': {}
                },
                'fusion_output_dim': TRAINING_CONFIG['projection_dim'] + 64,
                'dropout_rate': TRAINING_CONFIG['dropout_rate']
            },
            'training': {
                'batch_size': TRAINING_CONFIG['batch_size'],
                'learning_rate': TRAINING_CONFIG['learning_rate'],
                'epochs': TRAINING_CONFIG['epochs'],
                'optimizer': 'adam',
                'scheduler': 'cosine',
                'weight_decay': TRAINING_CONFIG['weight_decay'],
                'label_smoothing': TRAINING_CONFIG['label_smoothing'],
                'gradient_clip_norm': TRAINING_CONFIG['gradient_clip_norm'],
                'early_stopping_patience': TRAINING_CONFIG['early_stopping_patience']
            },
            'device': traditional_config['device'],
            'seed': traditional_config['random_seed'],
            'output_dir': traditional_config['output_dir'],
            'save_checkpoints': True,
            'verbose': True
        }
        
        # 确保目录存在
        os.makedirs(os.path.dirname(CONFIG_PATH), exist_ok=True)
        
        # 保存配置文件
        with open(CONFIG_PATH, 'w', encoding='utf-8') as f:
            yaml.dump(default_shl_config, f, default_flow_style=False, allow_unicode=True)
        
        print(f"✓ 已创建默认配置文件: {CONFIG_PATH}")
    
    # 加载并验证配置
    try:
        with open(CONFIG_PATH, 'r', encoding='utf-8') as f:
            loaded_config = yaml.safe_load(f)
        
        # 验证必要的配置项
        required_sections = ['dataset', 'training', 'device']
        for section in required_sections:
            if section not in loaded_config:
                raise ValueError(f"配置文件缺少必要部分: {section}")
        
        print("✓ 配置文件验证通过")
        print(f"  数据集: {loaded_config['dataset']['name']}")
        print(f"  数据路径: {loaded_config['dataset']['path']}")
        print(f"  批次大小: {loaded_config['training']['batch_size']}")
        print(f"  学习率: {loaded_config['training']['learning_rate']}")
        
    except Exception as e:
        print(f"❌ 配置文件加载失败: {e}")
        print("   切换到传统模式...")
        USE_CONFIG_MODE = False

else:
    print("✓ 使用传统模式")
    if not USE_CONFIG_MODE:
        print("  原因: 手动设置为传统模式")
    elif ConfigBridge is None:
        print("  原因: ConfigBridge模块未可用")
    
    # 更新传统配置
    traditional_config.update(TRAINING_CONFIG)

# === 验证数据文件存在性 ===
print(f"\n验证SHL_Multimodal数据文件...")
clients_data_path = os.path.join(SHL_DATA_PATH, 'clientsData.hkl')
clients_label_path = os.path.join(SHL_DATA_PATH, 'clientsLabel.hkl')

if os.path.exists(clients_data_path) and os.path.exists(clients_label_path):
    print(f"✓ 数据文件存在:")
    print(f"  数据文件: {clients_data_path}")
    print(f"  标签文件: {clients_label_path}")
else:
    print(f"❌ 数据文件不存在:")
    print(f"  数据文件: {clients_data_path} ({'存在' if os.path.exists(clients_data_path) else '不存在'})")
    print(f"  标签文件: {clients_label_path} ({'存在' if os.path.exists(clients_label_path) else '不存在'})")
    print(f"  请确保SHL_Multimodal数据已正确处理并保存")

# === 设置全局变量 ===
dataset_name = traditional_config['dataset_name']
batch_size = traditional_config['batch_size']
learning_rate = traditional_config['learning_rate']
epochs = traditional_config['epochs']
random_seed = traditional_config['random_seed']
position_device = traditional_config['position_device']
main_dir = traditional_config['main_dir']

# === 最终配置摘要 ===
print(f"""
========== 🔥 配置摘要 ==========
模式: {'配置驱动' if USE_CONFIG_MODE else '传统模式'}
数据集: {dataset_name}
数据路径: {SHL_DATA_PATH}
训练参数:
  - 批次大小: {traditional_config['batch_size']}
  - 学习率: {traditional_config['learning_rate']}
  - 训练轮数: {traditional_config['epochs']}
  - 投影维度: {traditional_config['projection_dim']}
  - 注意力头数: {traditional_config['num_heads']}
  - 网络层数: {traditional_config['num_layers']}
输出目录: {traditional_config['output_dir']}
================================
""")

In [ ]:
# ========== 数据加载 - SHL_Multimodal数据集支持 ==========

print("开始加载SHL_Multimodal数据集...")

# === SHL_Multimodal数据集加载函数 ===
def load_shl_multimodal_dataset(data_path: str, config: Dict[str, Any]) -> Tuple[np.ndarray, np.ndarray]:
    """
    加载SHL_Multimodal数据集
    
    Args:
        data_path: 数据集路径 (应该是包含clientsData.hkl和clientsLabel.hkl的目录)
        config: 数据配置
    
    Returns:
        (combined_data, combined_labels)
    """
    import hickle as hkl
    
    # SHL_Multimodal数据集文件路径
    clients_data_path = os.path.join(data_path, 'clientsData.hkl')
    clients_label_path = os.path.join(data_path, 'clientsLabel.hkl')
    
    # 检查数据文件是否存在
    if not os.path.exists(clients_data_path) or not os.path.exists(clients_label_path):
        print(f"❌ SHL_Multimodal数据文件不存在:")
        print(f"   数据文件: {clients_data_path} ({'存在' if os.path.exists(clients_data_path) else '不存在'})")
        print(f"   标签文件: {clients_label_path} ({'存在' if os.path.exists(clients_label_path) else '不存在'})")
        print(f"   请确保已正确运行DATA_SHL_NEW.ipynb生成数据")
        raise FileNotFoundError("SHL_Multimodal数据文件不存在")
    
    try:
        # 加载数据
        print(f"正在加载SHL_Multimodal数据文件...")
        print(f"  从路径: {data_path}")
        
        clients_data = hkl.load(clients_data_path)
        clients_labels = hkl.load(clients_label_path)
        
        print(f"✓ 成功加载SHL_Multimodal数据")
        print(f"  客户端/流数量: {len(clients_data)}")
        print(f"  标签数量: {len(clients_labels)}")
        
        # 检查数据结构
        if len(clients_data) != len(clients_labels):
            raise ValueError(f"数据和标签数量不匹配: {len(clients_data)} vs {len(clients_labels)}")
        
        # 合并所有客户端/流数据
        all_data = []
        all_labels = []
        
        for i, (client_data, client_label) in enumerate(zip(clients_data, clients_labels)):
            if client_data is not None and len(client_data) > 0:
                # 确保数据是numpy数组
                if not isinstance(client_data, np.ndarray):
                    client_data = np.array(client_data)
                if not isinstance(client_label, np.ndarray):
                    client_label = np.array(client_label)
                
                all_data.append(client_data)
                all_labels.append(client_label)
                print(f"  客户端/流 {i}: {client_data.shape} (样本 x 特征)")
                print(f"    标签范围: {np.min(client_label)} - {np.max(client_label)}")
            else:
                print(f"  客户端/流 {i}: 跳过 (数据为空)")
        
        if not all_data:
            raise ValueError("没有有效的数据可加载")
        
        # 合并数据
        print(f"合并 {len(all_data)} 个客户端/流的数据...")
        combined_data = np.vstack(all_data)
        combined_labels = np.hstack(all_labels)
        
        print(f"✓ 数据合并完成")
        print(f"  总样本数: {combined_data.shape[0]:,}")
        print(f"  特征维度: {combined_data.shape[1]}")
        print(f"  数据类型: {combined_data.dtype}")
        print(f"  数据范围: [{np.min(combined_data):.3f}, {np.max(combined_data):.3f}]")
        print(f"  标签范围: {np.min(combined_labels)} - {np.max(combined_labels)}")
        print(f"  唯一标签: {sorted(np.unique(combined_labels))}")
        
        # 检查数据质量
        if np.any(np.isnan(combined_data)):
            nan_count = np.sum(np.isnan(combined_data))
            print(f"⚠️  发现 {nan_count} 个NaN值，将进行处理")
            combined_data = np.nan_to_num(combined_data, nan=0.0)
        
        if np.any(np.isinf(combined_data)):
            inf_count = np.sum(np.isinf(combined_data))
            print(f"⚠️  发现 {inf_count} 个无穷值，将进行处理")
            combined_data = np.nan_to_num(combined_data, posinf=0.0, neginf=0.0)
        
        return combined_data, combined_labels
        
    except Exception as e:
        print(f"❌ 加载SHL_Multimodal数据时出错: {e}")
        import traceback
        traceback.print_exc()
        raise

def split_shl_data(data: np.ndarray, labels: np.ndarray, 
                   train_ratio: float = 0.7, val_ratio: float = 0.15,
                   random_seed: int = 42) -> Tuple[np.ndarray, np.ndarray, np.ndarray, np.ndarray, np.ndarray, np.ndarray]:
    """
    分割SHL_Multimodal数据集
    
    Args:
        data: 输入数据
        labels: 标签
        train_ratio: 训练集比例
        val_ratio: 验证集比例
        random_seed: 随机种子
    
    Returns:
        (train_data, train_labels, val_data, val_labels, test_data, test_labels)
    """
    from sklearn.model_selection import train_test_split
    
    # 设置随机种子
    np.random.seed(random_seed)
    
    print(f"分割数据集 (train:{train_ratio:.1%}, val:{val_ratio:.1%}, test:{1-train_ratio-val_ratio:.1%})...")
    
    # 检查是否有足够的每个类别的样本进行分层抽样
    unique_labels, label_counts = np.unique(labels, return_counts=True)
    min_count = np.min(label_counts)
    
    print(f"标签分布:")
    activity_labels = shl_specific_config.get('activity_labels', [f'Class_{i}' for i in unique_labels])
    for label, count in zip(unique_labels, label_counts):
        label_name = activity_labels[label] if label < len(activity_labels) else f'Class_{label}'
        print(f"  {label_name}: {count:,} 样本")
    
    # 如果最少的类别样本数小于3，则不使用分层抽样
    use_stratify = min_count >= 3
    if not use_stratify:
        print(f"⚠️  最少类别样本数为 {min_count}，不使用分层抽样")
    
    try:
        # 第一次分割: 分离训练集和临时集(验证+测试)
        stratify_param = labels if use_stratify else None
        train_data, temp_data, train_labels, temp_labels = train_test_split(
            data, labels, test_size=(1-train_ratio), 
            random_state=random_seed, stratify=stratify_param
        )
        
        # 第二次分割: 从临时集中分离验证集和测试集
        val_ratio_adjusted = val_ratio / (1 - train_ratio)  # 调整验证集在临时集中的比例
        stratify_param_temp = temp_labels if use_stratify else None
        val_data, test_data, val_labels, test_labels = train_test_split(
            temp_data, temp_labels, test_size=(1-val_ratio_adjusted), 
            random_state=random_seed, stratify=stratify_param_temp
        )
        
    except ValueError as e:
        print(f"⚠️  分层抽样失败 ({e})，使用随机抽样")
        # 降级到随机抽样
        train_data, temp_data, train_labels, temp_labels = train_test_split(
            data, labels, test_size=(1-train_ratio), random_state=random_seed
        )
        val_ratio_adjusted = val_ratio / (1 - train_ratio)
        val_data, test_data, val_labels, test_labels = train_test_split(
            temp_data, temp_labels, test_size=(1-val_ratio_adjusted), random_state=random_seed
        )
    
    print(f"✓ 数据分割完成:")
    print(f"  训练集: {train_data.shape[0]:,} 样本 ({train_data.shape[0]/data.shape[0]*100:.1f}%)")
    print(f"  验证集: {val_data.shape[0]:,} 样本 ({val_data.shape[0]/data.shape[0]*100:.1f}%)")
    print(f"  测试集: {test_data.shape[0]:,} 样本 ({test_data.shape[0]/data.shape[0]*100:.1f}%)")
    
    return train_data, train_labels, val_data, val_labels, test_data, test_labels

# === 根据模式加载数据 ===
try:
    if USE_CONFIG_MODE and ConfigBridge is not None:
        # 配置驱动模式
        print("使用配置驱动模式加载数据...")
        
        # 创建训练器实例并加载数据
        trainer = ConfigurableTrainer(CONFIG_PATH)
        # 注意: 实际的数据加载逻辑需要在这里实现
        print("⚠️  配置驱动模式数据加载需要进一步实现")
        USE_CONFIG_MODE = False  # 临时切换到传统模式
        
    if not USE_CONFIG_MODE:
        # 传统模式
        print("使用传统模式加载数据...")
        
        # 加载SHL_Multimodal数据集
        all_data, all_labels = load_shl_multimodal_dataset(SHL_DATA_PATH, traditional_config)
        
        # 分割数据集
        train_split = shl_specific_config.get('train_split', 0.7)
        val_split = shl_specific_config.get('val_split', 0.15)
        random_seed = traditional_config.get('random_seed', 42)
        
        (central_train_data, central_train_label, 
         central_dev_data, central_dev_label, 
         central_test_data, central_test_label) = split_shl_data(
            all_data, all_labels, train_split, val_split, random_seed
        )
        
        print("✓ 传统模式数据加载完成")
    
    # === 数据后处理和验证 ===
    print(f"\n数据加载摘要:")
    print(f"训练集形状: {central_train_data.shape}")
    print(f"验证集形状: {central_dev_data.shape}")  
    print(f"测试集形状: {central_test_data.shape}")
    print(f"训练标签范围: {np.min(central_train_label)} - {np.max(central_train_label)}")
    print(f"唯一标签数量: {len(np.unique(central_train_label))}")
    
    # 活动标签映射 (SHL数据集)
    ACTIVITY_LABEL = shl_specific_config['activity_labels']
    activity_count = len(ACTIVITY_LABEL)
    
    print(f"\n活动类别 ({activity_count}个):")
    for i, activity in enumerate(ACTIVITY_LABEL):
        train_count = np.sum(central_train_label == i)
        val_count = np.sum(central_dev_label == i)
        test_count = np.sum(central_test_label == i)
        total_count = train_count + val_count + test_count
        print(f"  {i}: {activity}")
        print(f"     训练: {train_count:,}, 验证: {val_count:,}, 测试: {test_count:,}, 总计: {total_count:,}")
    
    # 验证数据完整性
    print(f"\n验证数据完整性...")
    assert central_train_data.shape[0] == central_train_label.shape[0], "训练数据和标签数量不匹配"
    assert central_dev_data.shape[0] == central_dev_label.shape[0], "验证数据和标签数量不匹配"
    assert central_test_data.shape[0] == central_test_label.shape[0], "测试数据和标签数量不匹配"
    
    # 检查标签范围
    all_labels_combined = np.concatenate([central_train_label, central_dev_label, central_test_label])
    unique_labels = np.unique(all_labels_combined)
    assert np.min(unique_labels) >= 0, "标签包含负值"
    assert np.max(unique_labels) < activity_count, f"标签超出范围 (最大: {np.max(unique_labels)}, 期望 < {activity_count})"
    
    # 检查数据维度一致性
    expected_features = central_train_data.shape[1]
    assert central_dev_data.shape[1] == expected_features, f"验证集特征维度不匹配: {central_dev_data.shape[1]} vs {expected_features}"
    assert central_test_data.shape[1] == expected_features, f"测试集特征维度不匹配: {central_test_data.shape[1]} vs {expected_features}"
    
    print("✓ 数据完整性验证通过")
    
    # 数据统计信息
    print(f"\n数据统计信息:")
    print(f"  特征维度: {expected_features}")
    print(f"  数据类型: {central_train_data.dtype}")
    print(f"  训练数据范围: [{np.min(central_train_data):.3f}, {np.max(central_train_data):.3f}]")
    print(f"  训练数据均值: {np.mean(central_train_data):.3f}")
    print(f"  训练数据标准差: {np.std(central_train_data):.3f}")
    
    # 设置兼容性变量 (与原始代码保持一致)
    client_orientation_train = None
    client_orientation_test = None  
    orientations_names = None
    position_device = traditional_config.get('position_device', '')
    
    print("✓ SHL_Multimodal数据集加载完成!")

except Exception as e:
    print(f"❌ 数据加载失败: {e}")
    import traceback
    traceback.print_exc()
    
    # 如果数据加载失败，生成示例数据以便测试
    print(f"\n⚠️  数据加载失败，生成示例数据以便测试...")
    print(f"请检查以下问题:")
    print(f"1. 数据路径是否正确: {SHL_DATA_PATH}")
    print(f"2. 是否存在 clientsData.hkl 和 clientsLabel.hkl 文件")
    print(f"3. 是否已运行 DATA_SHL_NEW.ipynb 生成数据")
    
    # 生成示例SHL_Multimodal数据
    n_train = 1000
    n_val = 200
    n_test = 300
    
    # 根据SHL_Multimodal的实际特征维度设置
    # 假设包含多个传感器模态的特征
    n_features = 128 * 18  # 128个时间步 * 18个传感器通道 (Acc(3) + Gyr(3) + Mag(3) + LAcc(3) + Gra(3) + Ori(3))
    n_classes = len(shl_specific_config['activity_labels'])
    
    np.random.seed(42)
    
    print(f"生成示例数据:")
    print(f"  特征维度: {n_features} (假设 128时间步 × 18传感器通道)")
    print(f"  类别数: {n_classes}")
    
    central_train_data = np.random.randn(n_train, n_features).astype(np.float32)
    central_train_label = np.random.randint(0, n_classes, n_train)
    
    central_dev_data = np.random.randn(n_val, n_features).astype(np.float32)
    central_dev_label = np.random.randint(0, n_classes, n_val)
    
    central_test_data = np.random.randn(n_test, n_features).astype(np.float32)
    central_test_label = np.random.randint(0, n_classes, n_test)
    
    # 设置活动标签和计数
    ACTIVITY_LABEL = shl_specific_config['activity_labels']
    activity_count = len(ACTIVITY_LABEL)
    
    # 兼容性变量
    client_orientation_train = None
    client_orientation_test = None
    orientations_names = None
    position_device = ''
    
    print(f"✓ 示例数据生成完成:")
    print(f"  训练集: {central_train_data.shape}")
    print(f"  验证集: {central_dev_data.shape}")
    print(f"  测试集: {central_test_data.shape}")
    print(f"  类别数: {activity_count}")

print(f"\n{'='*50}")
print("数据加载阶段完成")
print(f"{'='*50}")

# === 数据加载后的额外检查 ===
print(f"\n最终数据检查:")
print(f"数据集类型: SHL_Multimodal")
print(f"数据来源: {SHL_DATA_PATH}")
print(f"训练样本: {central_train_data.shape[0]:,}")
print(f"验证样本: {central_dev_data.shape[0]:,}")
print(f"测试样本: {central_test_data.shape[0]:,}")
print(f"特征维度: {central_train_data.shape[1]:,}")
print(f"活动类别: {activity_count}个")
print(f"数据类型: {central_train_data.dtype}")

# 检查内存使用
train_size_mb = central_train_data.nbytes / (1024 * 1024)
total_size_mb = (central_train_data.nbytes + central_dev_data.nbytes + central_test_data.nbytes) / (1024 * 1024)
print(f"内存使用: 训练集 {train_size_mb:.1f}MB, 总计 {total_size_mb:.1f}MB")

if total_size_mb > 1000:  # 超过1GB时警告
    print(f"⚠️  数据集较大 ({total_size_mb:.1f}MB)，建议使用较小的批次大小")
    if traditional_config['batch_size'] > 32:
        traditional_config['batch_size'] = 32
        print(f"   自动调整批次大小为: {traditional_config['batch_size']}")

In [ ]:
# ========== 数据预处理和数据加载器创建 ==========

print("开始训练准备...")

try:
    # 如果在RealWorld或HHAR上使用指定位置/设备，我们移除一个并将其用作测试集，并将其他用于训练
    # SHL数据集不需要此步骤，但保留兼容性
    if position_device != '' or dataset_name == 'UCI':
        print(f"注意: SHL数据集不支持位置设备分割 (position_device='{position_device}')")
        print("使用标准数据分割...")
    
    # 计算类权重 (处理不平衡数据集)
    print("计算类权重...")
    from sklearn.utils import class_weight
    
    temp_weights = class_weight.compute_class_weight(
        class_weight='balanced',
        classes=np.unique(central_train_label),
        y=central_train_label.ravel()
    )
    class_weights = {j: temp_weights[j] for j in range(len(temp_weights))}
    
    print("✓ 类权重计算完成:")
    for i, weight in class_weights.items():
        activity_name = ACTIVITY_LABEL[i] if i < len(ACTIVITY_LABEL) else f"类别{i}"
        print(f"  {activity_name}: {weight:.3f}")

    # 创建PyTorch数据集和数据加载器
    print("创建PyTorch数据集...")
    
    # 创建特征和标签的张量
    train_features = torch.FloatTensor(central_train_data)
    train_labels = torch.LongTensor(central_train_label)
    dev_features = torch.FloatTensor(central_dev_data)
    dev_labels = torch.LongTensor(central_dev_label)
    test_features = torch.FloatTensor(central_test_data)
    test_labels = torch.LongTensor(central_test_label)

    # 创建数据集
    train_dataset = HARDataset(train_features, train_labels)
    dev_dataset = HARDataset(dev_features, dev_labels)
    test_dataset = HARDataset(test_features, test_labels)

    # 获取批次大小
    batch_size = traditional_config.get('batch_size', 32)

    # 创建数据加载器
    train_loader = torch.utils.data.DataLoader(
        train_dataset, batch_size=batch_size, shuffle=True, 
        num_workers=0, pin_memory=False, drop_last=False
    )
    dev_loader = torch.utils.data.DataLoader(
        dev_dataset, batch_size=batch_size, shuffle=False,
        num_workers=0, pin_memory=False, drop_last=False
    )
    test_loader = torch.utils.data.DataLoader(
        test_dataset, batch_size=batch_size, shuffle=False,
        num_workers=0, pin_memory=False, drop_last=False
    )

    print(f"✓ 数据加载器创建完成:")
    print(f"  训练批次数: {len(train_loader)}")
    print(f"  验证批次数: {len(dev_loader)}")
    print(f"  测试批次数: {len(test_loader)}")
    print(f"  批次大小: {batch_size}")

except Exception as e:
    print(f"❌ 数据预处理失败: {e}")
    import traceback
    traceback.print_exc()
    raise

print("✓ 数据预处理阶段完成")

In [ ]:
# ========== 模型创建和训练设置 ==========

print(f"{'='*60}")
print("开始模型训练")
print(f"{'='*60}")

try:
    # === 传统模式训练 ===
    print("使用传统训练模式...")
    
    # 获取训练参数
    learning_rate = traditional_config.get('learning_rate', 0.001)
    epochs = traditional_config.get('epochs', 100)
    weight_decay = traditional_config.get('weight_decay', 0.0001)
    random_seed = traditional_config.get('random_seed', 42)
    
    # 设置设备
    device_config = traditional_config.get('device', 'auto')
    if device_config == 'auto':
        if torch.cuda.is_available():
            device = torch.device('cuda')
            print(f"使用 CUDA 设备: {torch.cuda.get_device_name()}")
        elif torch.backends.mps.is_available():
            device = torch.device('mps')
            print("使用 MPS 设备")
        else:
            device = torch.device('cpu')
            print("使用 CPU 设备")
    else:
        device = torch.device(device_config)
        print(f"使用指定设备: {device}")
    
    # 创建模型
    input_dim = central_train_data.shape[1]  # 特征维度
    num_classes = len(np.unique(central_train_label))  # 类别数量
    
    print(f"创建模型:")
    print(f"  输入维度: {input_dim}")
    print(f"  输出类别数: {num_classes}")
    
    # 创建模型实例
    model_config = {
        'input_dim': input_dim,
        'num_classes': num_classes,
        'projection_dim': traditional_config.get('projection_dim', 192),
        'num_heads': traditional_config.get('num_heads', 4),
        'num_layers': traditional_config.get('num_layers', 3),
        'patch_size': traditional_config.get('patch_size', 16),
        'time_step': traditional_config.get('time_step', 16),
        'conv_kernel_size': traditional_config.get('conv_kernel_size', 15),
        'dropout_rate': traditional_config.get('dropout_rate', 0.1)
    }
    
    # 使用CBranchformer模型
    har_model = model.CBranchformerHAR(
        input_dim=model_config['input_dim'],
        num_classes=model_config['num_classes'],
        projection_dim=model_config['projection_dim'],
        num_heads=model_config['num_heads'],
        num_layers=model_config['num_layers'],
        patch_size=model_config['patch_size'],
        time_step=model_config['time_step'],
        conv_kernel_size=model_config['conv_kernel_size'],
        dropout_rate=model_config['dropout_rate']
    ).to(device)
    
    print(f"✓ 模型创建完成")
    print(f"  模型参数量: {sum(p.numel() for p in har_model.parameters()):,}")
    
    # 创建优化器和损失函数
    optimizer = torch.optim.Adam(
        har_model.parameters(), 
        lr=learning_rate, 
        weight_decay=weight_decay
    )
    
    # 使用加权交叉熵损失处理类别不平衡
    class_weights_tensor = torch.FloatTensor([class_weights[i] for i in range(num_classes)]).to(device)
    criterion = torch.nn.CrossEntropyLoss(weight=class_weights_tensor)
    
    # 学习率调度器
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=epochs)
    
    print(f"✓ 优化器和损失函数设置完成")
    print(f"  优化器: Adam (lr={learning_rate}, weight_decay={weight_decay})")
    print(f"  损失函数: 加权交叉熵")
    print(f"  学习率调度: Cosine Annealing")
    
    # 设置输出目录
    output_dir = Path(traditional_config.get('output_dir', './results/shl_multimodal'))
    output_dir.mkdir(parents=True, exist_ok=True)
    
    print(f"  输出目录: {output_dir}")
    
except Exception as e:
    print(f"❌ 模型创建失败: {e}")
    import traceback
    traceback.print_exc()
    raise

print("✓ 模型创建和训练设置完成")

In [ ]:
# ========== 训练循环执行 ==========

print(f"\n开始训练 (共{epochs}个epoch)...")

try:
    # 训练参数
    best_val_acc = 0.0
    best_val_f1 = 0.0
    patience = traditional_config.get('early_stopping_patience', 10)
    patience_counter = 0
    gradient_clip_norm = traditional_config.get('gradient_clip_norm', 1.0)
    
    # 记录训练历史
    train_losses = []
    train_accs = []
    val_losses = []
    val_accs = []
    val_f1s = []
    
    print(f"训练配置:")
    print(f"  设备: {device}")
    print(f"  批次大小: {batch_size}")
    print(f"  学习率: {learning_rate}")
    print(f"  训练轮数: {epochs}")
    print(f"  早停耐心: {patience}")
    print(f"  梯度裁剪: {gradient_clip_norm}")
    print(f"{'='*60}")
    
    for epoch in range(epochs):
        epoch_start_time = time.time()
        
        # === 训练阶段 ===
        har_model.train()
        train_loss = 0.0
        train_correct = 0
        train_total = 0
        
        for batch_idx, (data, targets) in enumerate(train_loader):
            data, targets = data.to(device), targets.to(device)
            
            optimizer.zero_grad()
            outputs = har_model(data)
            loss = criterion(outputs, targets)
            loss.backward()
            
            # 梯度裁剪
            if gradient_clip_norm > 0:
                torch.nn.utils.clip_grad_norm_(har_model.parameters(), gradient_clip_norm)
            
            optimizer.step()
            
            train_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            train_total += targets.size(0)
            train_correct += (predicted == targets).sum().item()
            
            # 打印批次进度 (每50个批次)
            if batch_idx % 50 == 0 and batch_idx > 0:
                current_acc = 100.0 * train_correct / train_total
                print(f"  Epoch [{epoch+1}/{epochs}] Batch [{batch_idx}/{len(train_loader)}] "
                      f"Loss: {loss.item():.4f} Acc: {current_acc:.2f}%")
        
        # === 验证阶段 ===
        har_model.eval()
        val_loss = 0.0
        val_correct = 0
        val_total = 0
        all_preds = []
        all_targets = []
        
        with torch.no_grad():
            for data, targets in dev_loader:
                data, targets = data.to(device), targets.to(device)
                outputs = har_model(data)
                loss = criterion(outputs, targets)
                
                val_loss += loss.item()
                _, predicted = torch.max(outputs.data, 1)
                val_total += targets.size(0)
                val_correct += (predicted == targets).sum().item()
                
                all_preds.extend(predicted.cpu().numpy())
                all_targets.extend(targets.cpu().numpy())
        
        # 计算指标
        train_acc = 100.0 * train_correct / train_total
        val_acc = 100.0 * val_correct / val_total
        val_f1 = f1_score(all_targets, all_preds, average='weighted') * 100
        
        train_losses.append(train_loss / len(train_loader))
        train_accs.append(train_acc)
        val_losses.append(val_loss / len(dev_loader))
        val_accs.append(val_acc)
        val_f1s.append(val_f1)
        
        # 更新学习率
        scheduler.step()
        current_lr = scheduler.get_last_lr()[0]
        
        # 计算epoch时间
        epoch_time = time.time() - epoch_start_time
        
        # 打印训练信息
        print(f"Epoch [{epoch+1:3d}/{epochs}] "
              f"Train Loss: {train_losses[-1]:.4f} "
              f"Train Acc: {train_acc:.2f}% "
              f"Val Loss: {val_losses[-1]:.4f} "
              f"Val Acc: {val_acc:.2f}% "
              f"Val F1: {val_f1:.2f}% "
              f"LR: {current_lr:.2e} "
              f"Time: {epoch_time:.1f}s")
        
        # 早停检查和模型保存
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            best_val_f1 = val_f1
            patience_counter = 0
            
            # 保存最佳模型
            if traditional_config.get('save_checkpoints', True):
                torch.save({
                    'epoch': epoch,
                    'model_state_dict': har_model.state_dict(),
                    'optimizer_state_dict': optimizer.state_dict(),
                    'scheduler_state_dict': scheduler.state_dict(),
                    'best_val_acc': best_val_acc,
                    'best_val_f1': best_val_f1,
                    'train_losses': train_losses,
                    'val_losses': val_losses,
                    'val_accs': val_accs,
                    'model_config': model_config,
                    'traditional_config': traditional_config
                }, output_dir / 'best_model.pth')
            
            print(f"    ✓ 新的最佳验证准确率: {best_val_acc:.2f}% (F1: {best_val_f1:.2f}%)")
        else:
            patience_counter += 1
            if patience_counter >= patience:
                print(f"\n早停触发! 验证准确率在{patience}个epoch内未改善")
                print(f"最佳验证准确率: {best_val_acc:.2f}%")
                print(f"最佳验证F1分数: {best_val_f1:.2f}%")
                break
        
        # 每10个epoch保存一次检查点
        if (epoch + 1) % 10 == 0 and traditional_config.get('save_checkpoints', True):
            checkpoint_path = output_dir / f'checkpoint_epoch_{epoch+1}.pth'
            torch.save({
                'epoch': epoch,
                'model_state_dict': har_model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'scheduler_state_dict': scheduler.state_dict(),
                'train_losses': train_losses,
                'val_losses': val_losses,
                'val_accs': val_accs
            }, checkpoint_path)
    
    print(f"\n✓ 训练完成!")
    print(f"  最佳验证准确率: {best_val_acc:.2f}%")
    print(f"  最佳验证F1分数: {best_val_f1:.2f}%")
    
except Exception as e:
    print(f"❌ 训练执行失败: {e}")
    import traceback
    traceback.print_exc()
    raise

print("✓ 训练循环执行完成")

In [ ]:
# ========== 测试评估 ==========

print(f"\n开始测试最佳模型...")

try:
    # 加载最佳模型
    if traditional_config.get('save_checkpoints', True):
        checkpoint_path = output_dir / 'best_model.pth'
        if checkpoint_path.exists():
            checkpoint = torch.load(checkpoint_path, map_location=device)
            har_model.load_state_dict(checkpoint['model_state_dict'])
            print("✓ 已加载最佳模型权重")
        else:
            print("⚠️  最佳模型文件不存在，使用当前模型")
    
    # 测试评估
    har_model.eval()
    test_correct = 0
    test_total = 0
    all_test_preds = []
    all_test_targets = []
    test_loss = 0.0
    
    with torch.no_grad():
        for data, targets in test_loader:
            data, targets = data.to(device), targets.to(device)
            outputs = har_model(data)
            loss = criterion(outputs, targets)
            
            test_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            
            test_total += targets.size(0)
            test_correct += (predicted == targets).sum().item()
            
            all_test_preds.extend(predicted.cpu().numpy())
            all_test_targets.extend(targets.cpu().numpy())
    
    test_acc = 100.0 * test_correct / test_total
    test_f1 = f1_score(all_test_targets, all_test_preds, average='weighted') * 100
    test_loss_avg = test_loss / len(test_loader)
    
    print(f"✓ 测试完成!")
    print(f"  测试损失: {test_loss_avg:.4f}")
    print(f"  测试准确率: {test_acc:.2f}%")
    print(f"  测试F1分数: {test_f1:.2f}%")
    
    # 生成详细分类报告
    print(f"\n详细分类报告:")
    class_names = [ACTIVITY_LABEL[i] if i < len(ACTIVITY_LABEL) else f"类别{i}" 
                  for i in range(num_classes)]
    report = classification_report(
        all_test_targets, all_test_preds, 
        target_names=class_names, 
        digits=3
    )
    print(report)
    
    # 生成混淆矩阵
    print(f"\n混淆矩阵:")
    cm = confusion_matrix(all_test_targets, all_test_preds)
    print(cm)
    
    # 计算每个类别的准确率
    print(f"\n各类别准确率:")
    for i in range(num_classes):
        class_mask = np.array(all_test_targets) == i
        if np.sum(class_mask) > 0:
            class_acc = np.mean(np.array(all_test_preds)[class_mask] == i) * 100
            activity_name = ACTIVITY_LABEL[i] if i < len(ACTIVITY_LABEL) else f"类别{i}"
            print(f"  {activity_name}: {class_acc:.2f}% ({np.sum(class_mask)} 样本)")
    
    # 保存结果
    results = {
        'dataset': dataset_name,
        'model_config': model_config,
        'training_config': traditional_config,
        'best_val_acc': best_val_acc,
        'best_val_f1': best_val_f1,
        'test_loss': test_loss_avg,
        'test_acc': test_acc,
        'test_f1': test_f1,
        'confusion_matrix': cm.tolist(),
        'training_history': {
            'train_losses': train_losses,
            'train_accs': train_accs,
            'val_losses': val_losses,
            'val_accs': val_accs,
            'val_f1s': val_f1s
        }
    }
    
    # 保存结果到JSON文件
    with open(output_dir / 'training_results.json', 'w', encoding='utf-8') as f:
        json.dump({k: v for k, v in results.items() if k != 'confusion_matrix'}, 
                 f, indent=2, default=str, ensure_ascii=False)
    
    # 保存分类报告
    with open(output_dir / 'classification_report.txt', 'w', encoding='utf-8') as f:
        f.write(report)
    
    # 保存混淆矩阵
    np.save(output_dir / 'confusion_matrix.npy', cm)
    
    print(f"✓ 结果已保存到: {output_dir}")
    
except Exception as e:
    print(f"❌ 测试评估失败: {e}")
    import traceback
    traceback.print_exc()
    raise

print("✓ 测试评估完成")

In [ ]:
# ========== 训练结果可视化 ==========

print("生成训练结果可视化...")

try:
    # 设置matplotlib参数
    plt.rcParams['figure.figsize'] = (15, 10)
    plt.rcParams['font.size'] = 12
    
    # 创建子图
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    fig.suptitle(f'SHL数据集训练结果 - {dataset_name}', fontsize=16, fontweight='bold')
    
    # 1. 训练和验证损失
    axes[0, 0].plot(train_losses, label='训练损失', color='blue', linewidth=2)
    axes[0, 0].plot(val_losses, label='验证损失', color='red', linewidth=2)
    axes[0, 0].set_title('训练和验证损失')
    axes[0, 0].set_xlabel('Epoch')
    axes[0, 0].set_ylabel('损失')
    axes[0, 0].legend()
    axes[0, 0].grid(True, alpha=0.3)
    
    # 2. 训练和验证准确率
    axes[0, 1].plot(train_accs, label='训练准确率', color='blue', linewidth=2)
    axes[0, 1].plot(val_accs, label='验证准确率', color='red', linewidth=2)
    axes[0, 1].set_title('训练和验证准确率')
    axes[0, 1].set_xlabel('Epoch')
    axes[0, 1].set_ylabel('准确率 (%)')
    axes[0, 1].legend()
    axes[0, 1].grid(True, alpha=0.3)
    
    # 3. 验证F1分数
    axes[1, 0].plot(val_f1s, label='验证F1分数', color='green', linewidth=2)
    axes[1, 0].set_title('验证F1分数')
    axes[1, 0].set_xlabel('Epoch')
    axes[1, 0].set_ylabel('F1分数 (%)')
    axes[1, 0].legend()
    axes[1, 0].grid(True, alpha=0.3)
    
    # 4. 混淆矩阵热图
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names, ax=axes[1, 1])
    axes[1, 1].set_title('混淆矩阵')
    axes[1, 1].set_ylabel('真实标签')
    axes[1, 1].set_xlabel('预测标签')
    plt.setp(axes[1, 1].get_xticklabels(), rotation=45, ha="right", rotation_mode="anchor")

    # 调整布局
    plt.tight_layout()
    plt.subplots_adjust(top=0.93)
    
    # 保存图片
    plt.savefig(output_dir / 'training_results.png', dpi=300, bbox_inches='tight')
    plt.savefig(output_dir / 'training_results.pdf', bbox_inches='tight')
    
    print("✓ 训练结果可视化已保存")
    plt.show()
    
    # 生成类别准确率条形图
    plt.figure(figsize=(12, 6))
    class_accuracies = []
    for i in range(num_classes):
        class_mask = np.array(all_test_targets) == i
        if np.sum(class_mask) > 0:
            class_acc = np.mean(np.array(all_test_preds)[class_mask] == i) * 100
            class_accuracies.append(class_acc)
        else:
            class_accuracies.append(0)
    
    bars = plt.bar(range(num_classes), class_accuracies, 
                   color=plt.cm.Set3(np.linspace(0, 1, num_classes)))
    plt.title('各活动类别测试准确率', fontsize=14, fontweight='bold')
    plt.xlabel('活动类别')
    plt.ylabel('准确率 (%)')
    plt.xticks(range(num_classes), class_names, rotation=45, ha='right')
    plt.grid(True, alpha=0.3)
    
    # 在条形图上添加数值
    for bar in bars:
        height = bar.get_height()
        plt.text(bar.get_x() + bar.get_width()/2., height,
                f'{height:.1f}%', ha='center', va='bottom', fontsize=10)
    
    plt.ylim(0, 110)
    plt.tight_layout()
    plt.savefig(output_dir / 'class_accuracies.png', dpi=300, bbox_inches='tight')
    plt.savefig(output_dir / 'class_accuracies.pdf', bbox_inches='tight')
    
    print("✓ 类别准确率图已保存")
    plt.show()
    
except Exception as e:
    print(f"❌ 可视化生成失败: {e}")
    import traceback
    traceback.print_exc()

print("✓ 训练结果可视化完成")

In [ ]:
# ========== 训练总结和结果报告 ==========

print(f"\n{'='*60}")
print("🎉 SHL数据集训练流程完成!")
print(f"{'='*60}")

# === 训练配置总结 ===
print(f"\n📋 训练配置摘要:")
print(f"  数据集: {dataset_name}")
print(f"  数据路径: {SHL_DATA_PATH}")
print(f"  训练模式: {'配置驱动' if USE_CONFIG_MODE else '传统模式'}")
print(f"  设备: {device}")
print(f"  随机种子: {random_seed}")

# === 数据摘要 ===
print(f"\n📊 数据摘要:")
print(f"  训练样本: {central_train_data.shape[0]:,}")
print(f"  验证样本: {central_dev_data.shape[0]:,}")
print(f"  测试样本: {central_test_data.shape[0]:,}")
print(f"  特征维度: {central_train_data.shape[1]:,}")
print(f"  活动类别: {activity_count}个")
print(f"  数据类型: {central_train_data.dtype}")

# === 模型配置摘要 ===
print(f"\n🧠 模型配置摘要:")
print(f"  模型类型: CBranchformer")
print(f"  参数量: {sum(p.numel() for p in har_model.parameters()):,}")
print(f"  投影维度: {model_config['projection_dim']}")
print(f"  注意力头数: {model_config['num_heads']}")
print(f"  网络层数: {model_config['num_layers']}")
print(f"  补丁大小: {model_config['patch_size']}")
print(f"  时间步长: {model_config['time_step']}")
print(f"  卷积核大小: {model_config['conv_kernel_size']}")
print(f"  Dropout率: {model_config['dropout_rate']}")

# === 训练超参数摘要 ===
print(f"\n⚙️ 训练超参数摘要:")
print(f"  批次大小: {batch_size}")
print(f"  学习率: {learning_rate}")
print(f"  训练轮数: {epochs} (实际: {len(train_losses)})")
print(f"  权重衰减: {weight_decay}")
print(f"  梯度裁剪: {gradient_clip_norm}")
print(f"  早停耐心: {patience}")
print(f"  优化器: Adam")
print(f"  调度器: CosineAnnealingLR")
print(f"  损失函数: 加权交叉熵")

# === 训练结果摘要 ===
print(f"\n🏆 训练结果摘要:")
print(f"  最佳验证准确率: {best_val_acc:.2f}%")
print(f"  最佳验证F1分数: {best_val_f1:.2f}%")
print(f"  测试准确率: {test_acc:.2f}%")
print(f"  测试F1分数: {test_f1:.2f}%")
print(f"  测试损失: {test_loss_avg:.4f}")

# === 性能分析 ===
print(f"\n📈 性能分析:")
if test_acc >= 90:
    print(f"  🟢 优秀: 测试准确率达到 {test_acc:.2f}%")
elif test_acc >= 80:
    print(f"  🟡 良好: 测试准确率为 {test_acc:.2f}%")
elif test_acc >= 70:
    print(f"  🟠 一般: 测试准确率为 {test_acc:.2f}%")
else:
    print(f"  🔴 待改进: 测试准确率仅为 {test_acc:.2f}%")

# 过拟合检查
train_val_gap = train_accs[-1] - val_accs[-1]
if train_val_gap > 10:
    print(f"  ⚠️ 可能过拟合: 训练准确率比验证准确率高 {train_val_gap:.1f}%")
    print(f"     建议: 增加dropout率、减少模型complexity或增加数据")
elif train_val_gap < -5:
    print(f"  ⚠️ 可能欠拟合: 验证准确率比训练准确率高 {abs(train_val_gap):.1f}%")
    print(f"     建议: 增加模型复杂度或减少正则化")
else:
    print(f"  ✅ 拟合良好: 训练验证准确率差距为 {train_val_gap:.1f}%")

# === 最佳表现类别分析 ===
print(f"\n🎯 各类别性能分析:")
class_perfs = []
for i in range(num_classes):
    class_mask = np.array(all_test_targets) == i
    if np.sum(class_mask) > 0:
        class_acc = np.mean(np.array(all_test_preds)[class_mask] == i) * 100
        activity_name = ACTIVITY_LABEL[i] if i < len(ACTIVITY_LABEL) else f"类别{i}"
        class_perfs.append((activity_name, class_acc, np.sum(class_mask)))

# 按准确率排序
class_perfs.sort(key=lambda x: x[1], reverse=True)

print(f"  最佳表现: {class_perfs[0][0]} ({class_perfs[0][1]:.1f}%)")
print(f"  最差表现: {class_perfs[-1][0]} ({class_perfs[-1][1]:.1f}%)")
print(f"  平均准确率: {np.mean([perf[1] for perf in class_perfs]):.1f}%")

# === 文件输出摘要 ===
print(f"\n📁 输出文件摘要:")
output_files = [
    'best_model.pth',
    'training_results.json', 
    'classification_report.txt',
    'confusion_matrix.npy',
    'training_results.png',
    'training_results.pdf',
    'class_accuracies.png',
    'class_accuracies.pdf'
]

for file in output_files:
    file_path = output_dir / file
    if file_path.exists():
        file_size = file_path.stat().st_size / 1024  # KB
        print(f"  ✅ {file} ({file_size:.1f}KB)")
    else:
        print(f"  ❌ {file} (未生成)")

print(f"\n  📂 所有结果保存在: {output_dir}")

# === 改进建议 ===
print(f"\n💡 改进建议:")
if test_acc < 85:
    print(f"  • 尝试增加训练轮数或调整学习率")
    print(f"  • 尝试不同的模型架构参数组合")
    print(f"  • 检查数据质量和标签准确性")
    print(f"  • 考虑数据增强技术")

if len(train_losses) == epochs:
    print(f"  • 训练可能未收敛，考虑增加训练轮数")

if total_size_mb > 500:
    print(f"  • 数据集较大，考虑使用更大的批次大小以提高效率")

print(f"\n🔄 下次训练时，只需修改第一个配置cell中的参数即可!")
print(f"\n{'='*60}")
print("训练流程全部完成! 🎉")
print(f"{'='*60}")